Importing Required Libraries

In [1]:
from email import message
import socket
import matplotlib.pyplot as plt
import json
import pandas as pd
import ast
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
from pyspark.sql.functions import split
from datetime import datetime
import plotly.express as px
import plotly.io as pio

In [11]:
#taking the server name and port name
HOST = '127.0.0.1'
PORT = 22223

In [12]:
#creating variables to load our variables from json
Timestamp=[]
Longitude=[]
Latitude=[]

#creating a socket at client side
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
df=pd.DataFrame()
#connecting and fetching data from the api that we started by producer
s.connect((HOST, PORT))
msg = s.recv(1024)

#fetching and printing data of api
while msg:
    row=ast.literal_eval(msg.decode("UTF-8"))
    msg = s.recv(1024)
    print(row)
    Timestamp.append(row['timestamp'])
    Latitude.append(float(row['iss_position']['latitude']))
    Longitude.append(float(row['iss_position']['longitude']))
    
s.close()


    

ConnectionRefusedError: [Errno 111] Connection refused

In [13]:
#creating a dataframe from the data we fetched
df= pd.DataFrame({'Timestamp': Timestamp,'Latitude': Latitude,'Longitude': Longitude})
df

,Timestamp,Latitude,Longitude


In [14]:
df.dtypes

Timestamp    float64
Latitude     float64
Longitude    float64
dtype: object

In [15]:
#converting timestamp to date and time format readable by users
df['Timestamp'] =  pd.to_datetime(df['Timestamp'], unit='s')
df['Time'] = df['Timestamp'].dt.time

In [16]:
df

,Timestamp,Latitude,Longitude,Time


In [18]:
#Plotting the graph
fig = px.scatter_geo(df,lat='Latitude',lon='Longitude', hover_name="Timestamp", projection="fahey")
fig.update_layout(title = 'ISS Location for 1 Hour', title_x=0.5, height=1000, width=1000)
pio.renderers.default = 'iframe'

fig.show()

#exporting the figure to local
fig.write_image("iss_60min.png")
fig.write_html("iss_60min.html")